In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import wrds

from dotenv import dotenv_values
config = dotenv_values(".env")

from mylib.load_from_wrds import query_options_stock, query_zero_curve

%load_ext autoreload
%autoreload 2

# Search and merge unique identifier (this section can be deleted later)

In [7]:
db = wrds.Connection(wrds_username=config['WRDS_USER'])
df_ID = db.raw_sql("SELECT secid, effect_date, cusip, ticker, class, issuer, issue, sic FROM optionm.secnmd", date_cols=['effect_date']) 

print(df_ID.shape)
df_ID.to_csv("./data/secnmd.csv", index = False)
df_ID.head()

(262899, 8)


secid effect_date     cusip ticker class                      issuer  \
0  5001.0  1996-01-02  00078110  ABSIE  None          ABS INDUSTRIES INC   
1  5001.0  2007-03-08  00078110   ZZZZ  None                ABS INDS INC   
2  5002.0  1996-01-01  00103010  AELNA  None       AEL INDUSTRIES - CI A   
3  5003.0  1996-01-01  00103810   AFAP  None  AFA PROTECTIVE SYSTEMS INC   
4  5003.0  1999-07-08  00103810  AFAPE  None  AFA PROTECTIVE SYSTEMS INC   

  issue   sic  
0  None  None  
1   COM  3462  
2  None  None  
3  None  None  
4  None  None

In [24]:
# for check purpose:
# pd.set_option('display.max_rows', 180)
df_ID.loc[df_ID['issuer'].str.contains("AIRSPAN"), :]

# df_ID_notna = df_ID.loc[df_ID['ticker'].notna(), :]
# df_ID_notna.loc[df_ID_notna['ticker'].str.contains("FREI"), :]

secid effect_date     cusip ticker class  \
14656   101174.0  2000-07-20  00950H10   AIRN  None   
14657   101174.0  2000-11-28  00950H10   AIRN  None   
14658   101174.0  2006-06-30  00950H10   AIRN  None   
257854  214944.0  2021-08-16  00951K10   MIMO  None   

                              issuer            issue   sic  
14656           AIRSPAN NETWORKS INC              COM  None  
14657           AIRSPAN NETWORKS INC              COM  4812  
14658           AIRSPAN NETWORKS INC              COM  3663  
257854  AIRSPAN NETWORKS HOLDINGS IN  ORDINARY SHARES  3663

In [8]:
df_firms = pd.read_excel("./data/firms.xlsx", sheet_name = "dotcom_firms")
df_firms.head()

Name Ticker  SecurityID  \
0                                               3Com   COMS    111020.0   
1                                        360networks   TSIX    111030.0   
2       AboveNet (formerly Metromedia Fiber Network)   MFNX    107497.0   
3  Actua Corporation (formerly Internet Capital G...   ICGE    106314.0   
4                                   Airspan Networks   AIRO         NaN   

                                         Wiki_Link  \
0               https://en.wikipedia.org/wiki/3Com   
1        https://en.wikipedia.org/wiki/360networks   
2           https://en.wikipedia.org/wiki/AboveNet   
3  https://en.wikipedia.org/wiki/Actua_Corporation   
4   https://en.wikipedia.org/wiki/Airspan_Networks   

                                       During_Bubble  
0   Shares soared after announcing the corporate ...  
1   A fiber optic company that had a market capit...  
2   Its stock rose 32% on the day it announced a ...  
3   A company that invested in B2B e-commerce com...  
4   A wireless firm; in July 2000, its stock pric...

In [9]:
# search company name in df_ID in order to find the correct SecurityID
# check: 
df_sec1 = pd.merge(df_firms[['Name', 'Ticker', 'SecurityID']], df_ID, left_on = 'SecurityID', right_on = 'secid', how = 'left')
df_sec1

Name Ticker  SecurityID     secid effect_date     cusip ticker  \
0           3Com   COMS    111020.0  111020.0  1996-01-02  88553510   COMS   
1           3Com   COMS    111020.0  111020.0  2000-11-28  88553510   COMS   
2           3Com   COMS    111020.0  111020.0  2010-04-13  88553510   ZZZZ   
3    360networks   TSIX    111030.0  111030.0  2000-07-27  88575T20   TSIX   
4    360networks   TSIX    111030.0  111030.0  2001-07-05  88575T20  TSIXQ   
..           ...    ...         ...       ...         ...       ...    ...   
435       Yahoo!   YHOO    112180.0  112180.0  1996-04-12  98433210   YHOO   
436       Yahoo!   YHOO    112180.0  112180.0  2000-11-28  98433210   YHOO   
437       Yahoo!   YHOO    112180.0  112180.0  2017-06-19  02134610   AABA   
438       Yahoo!   YHOO    112180.0  112180.0  2019-10-07  02134610   AABA   
439       Yahoo!   YHOO    112180.0  112180.0  2019-10-11  02134610   AABA   

    class           issuer            issue   sic  
0    None        3COM CORP              COM  None  
1    None        3COM CORP              COM  3577  
2    None        3COM CORP              COM  3577  
3    None  360NETWORKS INC      SUB VTG SHS  1731  
4    None  360NETWORKS INC      SUB VTG SHS  1731  
..    ...              ...              ...   ...  
435  None        YAHOO INC              COM  None  
436  None        YAHOO INC              COM  7375  
437  None       ALTABA INC              COM  7373  
438  None       ALTABA INC  ORDINARY SHARES  7373  
439  None       ALTABA INC  ORDINARY SHARES  7373  

[440 rows x 11 columns]

# load data

In [28]:
# Load and clean the data
db = wrds.Connection(wrds_username=config['WRDS_USER'])

Loading library list...
Done


In [32]:
df_firms = pd.read_excel("./data/firms.xlsx", sheet_name = "dotcom_firms", usecols = ['Name', 'Ticker', 'SecurityID'])
df_firms = df_firms.loc[df_firms['SecurityID'].notna(),:]
df_firms['SecurityID'] = df_firms['SecurityID'].astype('int')
print(df_firms.shape)
df_firms.head()

(75, 3)


Name Ticker  SecurityID
0                                               3Com   COMS      111020
1                                        360networks   TSIX      111030
2       AboveNet (formerly Metromedia Fiber Network)   MFNX      107497
3  Actua Corporation (formerly Internet Capital G...   ICGE      106314
4                                   Airspan Networks   AIRN      101174

In [35]:
from mylib.load_from_wrds import query_options_stock, query_zero_curve

startdate = '1997-01-01'
enddate = '2003-12-31'
for secid in df_firms.SecurityID.values:
    df_option = query_options_stock(db, secid, startdate=startdate, enddate=enddate)
    df_option.to_csv(f"data/df_option_{secid}.csv", index = False)
    

111020 Option price is not available in year 1997
